# Ohio 2008 Presidential Elections: Data Cleaning & Preprocessing

**Goal:** Build a clean, analysis-ready county-level table for Ohio, 2008 by merging the presidential primary and presidential general election results, and then derive summary stats (party totals).

**Output**: A single CSV where each row is a county and columns include:

- Primary per-candidate vote counts (prefixed with `pri_`)
- General per-candidate vote counts (prefixed with `gen_`)
- Party totals: `rep_primary_total`, `dem_primary_total`, `rep_general_total`, `dem_general_total`, `lib_general_total`, `cst_general_total`, `soc_general_total`, `grn_general_total`, `ind_general_total`

**Last Updated**: 2025/10/01

## 0. Library Import

In [47]:
import re
import pandas as pd
import numpy as np
from pathlib import Path

## 1. Inputs & Parameters

Define raw file paths once here so the entire notebook is easy to rerun on another machine. If a path changes, we only update it here. We keep a single `OUTPUT_PATH` so all exports land in one known place.

In [48]:
# OH 2008 dataset path
PRIMARY_PATH = r"../../data/raw/2008/OH/20080304__oh__primary.csv"
GENERAL_PATH = r"../../data/raw/2008/OH/20081104__oh__general.csv"

# Output directory
OUTPUT_PATH  = r"../../data/processed/2008/OH/"

# Analysis parameters
DISPLAY_ROWS = 10   # Number of rows to display in dataframes

## 2. Load & Filter

We load primary and general datasets separately and immediately subset to the rows we truly need:

- Restrict `office` to 'President' to avoid mixing down-ballot contests

- Remove columns that are fully missing or irrelevant post-filter (e.g., a district column that’s empty for county-level rows)

### a. Primary Election Dataset

In [49]:
# Load primary data
primary_df = pd.read_csv(PRIMARY_PATH)
primary_df.head(DISPLAY_ROWS)

,county,candidate,votes,party,office,district
0,Butler,Hillary Clinton,2143,Democratic,President,1
1,Hamilton,Hillary Clinton,40123,Democratic,President,1
2,Butler,John Edwards,52,Democratic,President,1
3,Hamilton,John Edwards,816,Democratic,President,1
4,Butler,Barack Obama,1139,Democratic,President,1
5,Hamilton,Barack Obama,74021,Democratic,President,1
6,Adams,Hillary Clinton,2829,Democratic,President,2
7,Brown,Hillary Clinton,5487,Democratic,President,2
8,Clermont,Hillary Clinton,16488,Democratic,President,2
9,Hamilton,Hillary Clinton,19311,Democratic,President,2


In [50]:
# Different values in 'office' column
primary_df["office"].value_counts()

office
President               904
US Representative       477
State Representative    366
State Senate            139
Name: count, dtype: int64

In [51]:
# Only keep rows where 'office' is 'President'
primary_df = primary_df[primary_df["office"] == "President"]
primary_df.shape

(904, 6)

In [52]:
# Now, drop the "office" column as it's no longer needed. Also, drop the district column
primary_df = primary_df.drop(columns=["office", "district"]).reset_index(drop=True)
primary_df.head(DISPLAY_ROWS)

,county,candidate,votes,party
0,Butler,Hillary Clinton,2143,Democratic
1,Hamilton,Hillary Clinton,40123,Democratic
2,Butler,John Edwards,52,Democratic
3,Hamilton,John Edwards,816,Democratic
4,Butler,Barack Obama,1139,Democratic
5,Hamilton,Barack Obama,74021,Democratic
6,Adams,Hillary Clinton,2829,Democratic
7,Brown,Hillary Clinton,5487,Democratic
8,Clermont,Hillary Clinton,16488,Democratic
9,Hamilton,Hillary Clinton,19311,Democratic


In [53]:
# Unique parties in primary_df
primary_df["party"].value_counts()

party
Republican    565
Democratic    339
Name: count, dtype: int64

In [54]:
# Candidates in primary_df
primary_df["candidate"].value_counts()

candidate
Hillary Clinton    113
John Edwards       113
Barack Obama       113
Mike Huckabee      113
John McCain        113
Ron Paul           113
Mitt Romney        113
Fred Thompson      113
Name: count, dtype: int64

In [55]:
# Missing values count
primary_df.isnull().sum()

county       0
candidate    0
votes        0
party        0
dtype: int64

In [56]:
# Final look at the cleaned primary_df
primary_df.head(DISPLAY_ROWS)

,county,candidate,votes,party
0,Butler,Hillary Clinton,2143,Democratic
1,Hamilton,Hillary Clinton,40123,Democratic
2,Butler,John Edwards,52,Democratic
3,Hamilton,John Edwards,816,Democratic
4,Butler,Barack Obama,1139,Democratic
5,Hamilton,Barack Obama,74021,Democratic
6,Adams,Hillary Clinton,2829,Democratic
7,Brown,Hillary Clinton,5487,Democratic
8,Clermont,Hillary Clinton,16488,Democratic
9,Hamilton,Hillary Clinton,19311,Democratic


In [57]:
# Shape after preprocessing
primary_df.shape

(904, 4)

### b. General Election Dataset

In [58]:
# Load general data
general_df = pd.read_csv(GENERAL_PATH)
general_df.head(DISPLAY_ROWS)

,candidate,county,district,office,party,votes
0,Donald Allen,Adams,NaN,President,Independent,0
1,Donald Allen,Allen,NaN,President,Independent,0
2,Donald Allen,Ashland,NaN,President,Independent,0
3,Donald Allen,Ashtabula,NaN,President,Independent,0
4,Donald Allen,Athens,NaN,President,Independent,0
5,Donald Allen,Auglaize,NaN,President,Independent,0
6,Donald Allen,Belmont,NaN,President,Independent,0
7,Donald Allen,Brown,NaN,President,Independent,0
8,Donald Allen,Butler,NaN,President,Independent,0
9,Donald Allen,Carroll,NaN,President,Independent,0


In [59]:
# Different values in 'office' column
general_df["office"].value_counts()

office
President               1232
State Representative     302
US Representative        283
Attorney General         264
State Senate             130
Name: count, dtype: int64

In [60]:
# Only keep rows where 'office' is 'President'
general_df = general_df[general_df["office"] == "President"]
general_df.shape

(1232, 6)

In [61]:
# Now, drop the "office" column as it's no longer needed. Also, drop the district column
general_df = general_df.drop(columns=["office", "district"]).reset_index(drop=True)
general_df.head(DISPLAY_ROWS)

,candidate,county,party,votes
0,Donald Allen,Adams,Independent,0
1,Donald Allen,Allen,Independent,0
2,Donald Allen,Ashland,Independent,0
3,Donald Allen,Ashtabula,Independent,0
4,Donald Allen,Athens,Independent,0
5,Donald Allen,Auglaize,Independent,0
6,Donald Allen,Belmont,Independent,0
7,Donald Allen,Brown,Independent,0
8,Donald Allen,Butler,Independent,0
9,Donald Allen,Carroll,Independent,0


In [62]:
# List out all the parties in the general election data
general_df["party"].value_counts()

party
Independent     704
Constitution     88
Libertarian      88
Republican       88
Green            88
Socialist        88
Democratic       88
Name: count, dtype: int64

In [63]:
# Missing values count
general_df.isnull().sum()

candidate    0
county       0
party        0
votes        0
dtype: int64

In [64]:
# Final look at cleaned general_df
general_df.head(DISPLAY_ROWS)

,candidate,county,party,votes
0,Donald Allen,Adams,Independent,0
1,Donald Allen,Allen,Independent,0
2,Donald Allen,Ashland,Independent,0
3,Donald Allen,Ashtabula,Independent,0
4,Donald Allen,Athens,Independent,0
5,Donald Allen,Auglaize,Independent,0
6,Donald Allen,Belmont,Independent,0
7,Donald Allen,Brown,Independent,0
8,Donald Allen,Butler,Independent,0
9,Donald Allen,Carroll,Independent,0


In [65]:
# Shape after preprocessing
general_df.shape

(1232, 4)

## 3. Table Pivoting

We convert tall (one row per county/party/candidate) into wide (one row per county with one column per candidate). This creates the consistent schema with previous group cleaned data.

Helper functions:

- `normalize_party(s)`: maps common forms (e.g., “Democratic”, “Republican”) to keys dem/rep so column names are stable
- `candidate_token(name)`: turns “Barack Obama” -> OBAMA, “John McCain” -> MCCAIN, etc. Create a short, readable, unique token for column names
- `pivot_wide(df, prefix, key_col="county")`: Main pivot function
        
    * groups by `county` x `party` × `candidate`, sums `votes`,
    * pivots to columns named like:
        * Primary: `pri_dem_OBAMA`, `pri_rep_MCCAIN`,...
        * General: `gen_dem_OBAMA`, `gen_rep_MCCAIN`,...

    * flattens the MultiIndex into plain column strings,
    * returns one wide row per county

In [66]:
def normalize_party(s: pd.Series) -> pd.Series:
    """
    Normalize party names: Democratic -> dem, Republican -> rep
    """
    return(s.str.strip()
           .str.capitalize()
           .map({
               "Democratic"     : "dem", 
               "Republican"     : "rep",
               "Libertarian"    : "lib",
               "Constitution"   : "cst",
               "Socialist"      : "soc",
               "Green"          : "grn",
               "Independent"    : "ind"
               ""
               })
           .fillna(s.str.strip().str.lower()))      # For defensive purposes only, would not expect other parties

In [82]:
SUFFIXES = {
    "JR","SR","JNR","SNR",
    "II","III","IV","V","VI","VII","VIII","IX","X","XI","XII"
}

def candidate_token(name: str) -> str:
    """
    Turn John McCain -> MCCAIN, Barack Obama -> OBAMA
    Skip suffixes, keep last name/token, capitalize, and remove punctuation
    """
    if pd.isna(name):
        return "UNKNOWN"                # Defensive purposes only, would not expect missing values
    
    # Remove suffixes
    raw = str(name).strip()

    # If a comma exists, treat as 'LAST, FIRST ...'
    if "," in raw:
        last_part = raw.split(",", 1)[0]
        last_part = re.sub(r"[^A-Za-z0-9\s]+", "", last_part).strip().upper()
        tokens = last_part.split()
        return tokens[-1] if tokens else "UNKNOWN"

    # Otherwise: remove punctuation, split, then drop trailing suffixes
    tokens = re.sub(r"[^A-Za-z0-9\s]+", "", raw).strip().upper().split()
    while tokens and tokens[-1] in SUFFIXES:
        tokens.pop()
    return tokens[-1] if tokens else "UNKNOWN"

In [68]:
def pivot_wide(df: pd.DataFrame, prefix: str, key_col: str="county") -> pd.DataFrame:
    """
    Pivot the dataframe to wide format based on party and candidate
    """
    # Normalize party names
    df['party_key'] = normalize_party(df['party'])
    
    # Create candidate tokens
    df['candidate_token'] = df['candidate'].apply(candidate_token)
    
    # Create new column names based on party and candidate token
    df['new_col'] = prefix + '_' + df['party'] + '_' + df['candidate_token']
    
    # Pivot the dataframe
    pivot_df = df.pivot_table(index=key_col, 
                              columns=["party_key", "candidate_token"], 
                              values="votes", 
                              aggfunc='sum', 
                              fill_value=0)
    
    # Flatten multi-level columns
    pivot_df.columns = [f"{prefix}_{p}_{c}" for p, c in pivot_df.columns]
    
    # Reset index to turn key_col back into a column
    pivot_df = pivot_df.reset_index()
    
    return pivot_df

In [69]:
# Primary dataframe pivot
primary_pivot = pivot_wide(primary_df, prefix="pri")
primary_pivot.head(DISPLAY_ROWS)

,county,pri_dem_CLINTON,pri_dem_EDWARDS,pri_dem_OBAMA,pri_rep_HUCKABEE,pri_rep_MCCAIN,pri_rep_PAUL,pri_rep_ROMNEY,pri_rep_THOMPSON
0,Adams,2829,145,829,1482,1492,75,104,67
1,Allen,7200,350,6801,4067,6813,848,738,323
2,Ashland,3913,153,2587,2687,3342,544,338,180
3,Ashtabula,13231,412,6375,2769,3818,682,329,214
4,Athens,7590,362,7396,1018,1771,408,108,71
5,Auglaize,3008,185,2085,1742,2615,490,282,124
6,Belmont,14204,932,4636,1283,2175,132,201,213
7,Brown,5487,277,1947,1945,2307,199,177,91
8,Butler,27599,589,22824,13758,19825,1179,1807,845
9,Carroll,3663,232,1731,1308,1839,275,166,103


In [70]:
# Primary dataframe shape after pivot
primary_pivot.shape

(88, 9)

In [71]:
# General dataframe pivot
general_pivot = pivot_wide(general_df, prefix="gen")
general_pivot.head(DISPLAY_ROWS)

,county,gen_cst_BALDWIN,gen_dem_OBAMA,gen_grn_MCKINNEY,gen_ind_ALLEN,gen_ind_DUNCAN,gen_ind_GERMALIC,gen_ind_KEYES,gen_ind_NADER,gen_ind_ROBERTSON,gen_ind_SCHRINER,gen_lib_BARR,gen_rep_MCCAIN,gen_soc_MOORE
0,Adams,51,4170,29,0,30,0,0,131,0,0,38,6914,25
1,Allen,123,19522,75,0,26,0,0,362,0,0,196,29940,19
2,Ashland,243,9300,76,0,31,0,0,225,0,1,117,15158,17
3,Ashtabula,129,25027,71,0,42,0,0,456,0,1,174,18949,25
4,Athens,109,20722,85,0,19,0,0,263,0,2,140,9742,16
5,Auglaize,53,6738,31,0,17,0,0,169,0,0,87,16414,7
6,Belmont,70,16302,52,0,32,0,1,402,0,1,104,15422,25
7,Brown,50,7503,32,0,26,0,0,220,0,0,84,12192,6
8,Butler,327,66030,222,0,104,0,6,1039,0,0,642,105341,66
9,Carroll,73,6423,45,0,25,0,0,213,0,0,64,7097,13


In [72]:
# General dataframe shape after pivot
general_pivot.shape

(88, 14)

## 4. Merge Dataframes

Before merging, we verify that county names match across primary and general:

In [73]:
# Check if county names match between primary_df and general_df
primary_counties = set(primary_df["county"].unique())
general_counties = set(general_df["county"].unique())
common_counties = primary_counties.intersection(general_counties)
print(f"Number of common counties: {len(common_counties)} out of {len(primary_counties)}")

Number of common counties: 88 out of 88


Great. Since we know that all counties name are matched, we don't need to perform further data preprocessing to match the county names. Thus, we can now merge them:

In [74]:
# Merge primary and general dataframes on 'county'
merged_df = primary_pivot.merge(general_pivot, on="county", how="inner").fillna(0)    # There should be no missing values to fill with 0
merged_df.head(DISPLAY_ROWS)

,county,pri_dem_CLINTON,pri_dem_EDWARDS,pri_dem_OBAMA,pri_rep_HUCKABEE,pri_rep_MCCAIN,pri_rep_PAUL,pri_rep_ROMNEY,pri_rep_THOMPSON,gen_cst_BALDWIN,...,gen_ind_ALLEN,gen_ind_DUNCAN,gen_ind_GERMALIC,gen_ind_KEYES,gen_ind_NADER,gen_ind_ROBERTSON,gen_ind_SCHRINER,gen_lib_BARR,gen_rep_MCCAIN,gen_soc_MOORE
0,Adams,2829,145,829,1482,1492,75,104,67,51,...,0,30,0,0,131,0,0,38,6914,25
1,Allen,7200,350,6801,4067,6813,848,738,323,123,...,0,26,0,0,362,0,0,196,29940,19
2,Ashland,3913,153,2587,2687,3342,544,338,180,243,...,0,31,0,0,225,0,1,117,15158,17
3,Ashtabula,13231,412,6375,2769,3818,682,329,214,129,...,0,42,0,0,456,0,1,174,18949,25
4,Athens,7590,362,7396,1018,1771,408,108,71,109,...,0,19,0,0,263,0,2,140,9742,16
5,Auglaize,3008,185,2085,1742,2615,490,282,124,53,...,0,17,0,0,169,0,0,87,16414,7
6,Belmont,14204,932,4636,1283,2175,132,201,213,70,...,0,32,0,1,402,0,1,104,15422,25
7,Brown,5487,277,1947,1945,2307,199,177,91,50,...,0,26,0,0,220,0,0,84,12192,6
8,Butler,27599,589,22824,13758,19825,1179,1807,845,327,...,0,104,0,6,1039,0,0,642,105341,66
9,Carroll,3663,232,1731,1308,1839,275,166,103,73,...,0,25,0,0,213,0,0,64,7097,13


In [75]:
# Statistics check on merged dataframe 
merged_df.describe()

,pri_dem_CLINTON,pri_dem_EDWARDS,pri_dem_OBAMA,pri_rep_HUCKABEE,pri_rep_MCCAIN,pri_rep_PAUL,pri_rep_ROMNEY,pri_rep_THOMPSON,gen_cst_BALDWIN,gen_dem_OBAMA,...,gen_ind_ALLEN,gen_ind_DUNCAN,gen_ind_GERMALIC,gen_ind_KEYES,gen_ind_NADER,gen_ind_ROBERTSON,gen_ind_SCHRINER,gen_lib_BARR,gen_rep_MCCAIN,gen_soc_MOORE
count,88.000000,88.000000,88.000000,88.000000,88.000000,88.000000,88.000000,88.000000,88.000000,88.000000,...,88.000000,88.000000,88.000000,88.000000,88.000000,88.000000,88.000000,88.000000,88.000000,88.000000
mean,14313.863636,446.954545,11997.375000,3850.431818,6482.420455,632.840909,619.772727,296.625000,142.784091,33409.590909,...,2.431818,44.375000,0.011364,1.818182,481.102273,0.715909,0.806818,226.329545,30429.772727,31.079545
std,23125.139942,551.272299,27847.418446,4287.015541,8787.733791,788.487464,906.853379,376.866369,167.622006,67712.943302,...,18.365257,46.562605,0.106600,5.553389,603.695345,6.715823,2.585805,331.002265,41173.882455,32.943777
min,1487.000000,72.000000,458.000000,351.000000,397.000000,39.000000,36.000000,34.000000,8.000000,2463.000000,...,0.000000,9.000000,0.000000,0.000000,94.000000,0.000000,0.000000,21.000000,3021.000000,3.000000
25%,3495.750000,158.500000,1788.000000,1570.250000,2232.500000,199.000000,174.500000,106.000000,51.750000,6504.750000,...,0.000000,17.000000,0.000000,0.000000,166.000000,0.000000,0.000000,70.000000,9605.750000,12.000000
50%,5482.500000,249.000000,2888.500000,2398.500000,3211.500000,324.000000,275.500000,162.000000,74.000000,11663.000000,...,0.000000,27.000000,0.000000,0.000000,252.500000,0.000000,0.000000,105.000000,15418.500000,18.500000
75%,14060.000000,449.000000,9030.250000,4206.000000,6324.000000,789.000000,708.750000,327.000000,154.750000,26607.750000,...,0.000000,48.500000,0.000000,1.000000,489.250000,0.000000,0.000000,225.250000,33471.250000,32.500000
max,160053.000000,3512.000000,190902.000000,23929.000000,47142.000000,4594.000000,5153.000000,2468.000000,900.000000,458422.000000,...,171.000000,233.000000,1.000000,41.000000,3616.000000,63.000000,21.000000,2128.000000,218486.000000,157.000000


Now, we will add party totals columns: 

- Primary totals:
    * `rep_primary_total` = sum of all `pri_rep_*` columns
    * `dem_primary_total` = sum of all `pri_dem_*` columns

- General totals:
    * `rep_general_total` = sum of all `gen_rep_*` columns
    * `dem_general_total` = sum of all `gen_dem_*` columns
    * `lib_general_total` = sum of all `gen_lib_*` columns
    * `cst_general_total` = sum of all `gen_cst_*` columns
    * `soc_general_total` = sum of all `gen_soc_*` columns
    * `grn_general_total` = sum of all `gen_grn_*` columns
    * `ind_general_total` = sum of all `gen_ind_*` columns

In [76]:
# Add party totals for primary election
rep_primary_cols   = [c for c in merged_df.columns if c.startswith("pri_rep_")]
dem_primary_cols   = [c for c in merged_df.columns if c.startswith("pri_dem_")]

merged_df["rep_primary_total"] = merged_df[rep_primary_cols].sum(axis=1) if rep_primary_cols else 0
merged_df["dem_primary_total"] = merged_df[dem_primary_cols].sum(axis=1) if dem_primary_cols else 0

In [77]:
# Add party totals for general election
rep_general_cols    = [c for c in merged_df.columns if c.startswith("gen_rep")] 
dem_general_cols    = [c for c in merged_df.columns if c.startswith("gen_dem")]
lib_general_cols    = [c for c in merged_df.columns if c.startswith("gen_lib")]
cst_general_cols    = [c for c in merged_df.columns if c.startswith("gen_cst")]
soc_general_cols    = [c for c in merged_df.columns if c.startswith("gen_soc")]
grn_general_cols    = [c for c in merged_df.columns if c.startswith("gen_grn")]
ind_general_cols    = [c for c in merged_df.columns if c.startswith("gen_ind")]

merged_df["rep_general_total"] = merged_df[rep_general_cols].sum(axis=1) if rep_general_cols else 0
merged_df["dem_general_total"] = merged_df[dem_general_cols].sum(axis=1) if dem_general_cols else 0
merged_df["lib_general_total"] = merged_df[lib_general_cols].sum(axis=1) if lib_general_cols else 0
merged_df["cst_general_total"] = merged_df[cst_general_cols].sum(axis=1) if cst_general_cols else 0
merged_df["soc_general_total"] = merged_df[soc_general_cols].sum(axis=1) if soc_general_cols else 0
merged_df["grn_general_total"] = merged_df[grn_general_cols].sum(axis=1) if grn_general_cols else 0
merged_df["ind_general_total"] = merged_df[ind_general_cols].sum(axis=1) if ind_general_cols else 0

In [81]:
# Print out all the column names in the final dataframe
print("Final columns in the cleaned dataframe:")
merged_df.columns

Final columns in the cleaned dataframe:


Index(['county', 'pri_dem_CLINTON', 'pri_dem_EDWARDS', 'pri_dem_OBAMA',
       'pri_rep_HUCKABEE', 'pri_rep_MCCAIN', 'pri_rep_PAUL', 'pri_rep_ROMNEY',
       'pri_rep_THOMPSON', 'gen_cst_BALDWIN', 'gen_dem_OBAMA',
       'gen_grn_MCKINNEY', 'gen_ind_ALLEN', 'gen_ind_DUNCAN',
       'gen_ind_GERMALIC', 'gen_ind_KEYES', 'gen_ind_NADER',
       'gen_ind_ROBERTSON', 'gen_ind_SCHRINER', 'gen_lib_BARR',
       'gen_rep_MCCAIN', 'gen_soc_MOORE', 'rep_primary_total',
       'dem_primary_total', 'rep_general_total', 'dem_general_total',
       'lib_general_total', 'cst_general_total', 'soc_general_total',
       'grn_general_total', 'ind_general_total'],
      dtype='object')

In [78]:
# Preview merged dataframe with totals
merged_df.head(DISPLAY_ROWS)

,county,pri_dem_CLINTON,pri_dem_EDWARDS,pri_dem_OBAMA,pri_rep_HUCKABEE,pri_rep_MCCAIN,pri_rep_PAUL,pri_rep_ROMNEY,pri_rep_THOMPSON,gen_cst_BALDWIN,...,gen_soc_MOORE,rep_primary_total,dem_primary_total,rep_general_total,dem_general_total,lib_general_total,cst_general_total,soc_general_total,grn_general_total,ind_general_total
0,Adams,2829,145,829,1482,1492,75,104,67,51,...,25,3220,3803,6914,4170,38,51,25,29,161
1,Allen,7200,350,6801,4067,6813,848,738,323,123,...,19,12789,14351,29940,19522,196,123,19,75,388
2,Ashland,3913,153,2587,2687,3342,544,338,180,243,...,17,7091,6653,15158,9300,117,243,17,76,257
3,Ashtabula,13231,412,6375,2769,3818,682,329,214,129,...,25,7812,20018,18949,25027,174,129,25,71,499
4,Athens,7590,362,7396,1018,1771,408,108,71,109,...,16,3376,15348,9742,20722,140,109,16,85,284
5,Auglaize,3008,185,2085,1742,2615,490,282,124,53,...,7,5253,5278,16414,6738,87,53,7,31,186
6,Belmont,14204,932,4636,1283,2175,132,201,213,70,...,25,4004,19772,15422,16302,104,70,25,52,436
7,Brown,5487,277,1947,1945,2307,199,177,91,50,...,6,4719,7711,12192,7503,84,50,6,32,246
8,Butler,27599,589,22824,13758,19825,1179,1807,845,327,...,66,37414,51012,105341,66030,642,327,66,222,1149
9,Carroll,3663,232,1731,1308,1839,275,166,103,73,...,13,3691,5626,7097,6423,64,73,13,45,238


In [80]:
# Save the cleaned and merged dataframe to CSV
out_dir = Path(OUTPUT_PATH)
out_dir.mkdir(parents=True, exist_ok=True)
merged_df.to_csv(OUTPUT_PATH + "OH.csv", index=False)